# Basics of Mobile Robotics Project

Autumn **2025-2026**

Group n°37:
- *Daniel Alves Ataìde - 340497*
- *Rachid Kahia - 343266*
- *Nicholas Thole - 356526*

## Introduction

### Project Objective

The objective of this project is to develop an autonomous mobile robot capable of navigating from a starting position to a goal position while avoiding obstacles in a controlled environment. The robot used is a Thymio II, equipped with infrared proximity sensors and controlled via the TDM protocol. A camera positioned above the workspace provides real-time visual feedback, enabling the robot to localize itself and detect obstacles.

The system handles two types of obstacles:

1. **Static obstacles (camera-detected)**: These are known obstacles placed in the environment that are detected by the overhead camera through color filtering. They are used during the global path planning phase to compute collision-free trajectories. These obstacles are not detectable by the robot's onboard sensors.

2. **Dynamic/unexpected obstacles (sensor-detected)**: These are obstacles that may appear during navigation and are detected by the Thymio's infrared proximity sensors. The robot must reactively avoid these obstacles using local navigation strategies while maintaining progress toward the goal.

In addition to basic navigation, the system must demonstrate robustness through two additional objectives:

1. **Vision-loss resilience**: If the camera is blocked or vision is temporarily lost, the robot must continue navigating toward the goal using dead reckoning based on wheel odometry. While precision will be reduced without visual feedback, the Kalman filter's prediction step allows the robot to maintain an estimated pose and continue making progress.

2. **Kidnapping recovery**: If the robot is suddenly moved to a new position (the "kidnapping problem"), the system must detect this significant change in pose, recompute the optimal path from the new location to the goal, and resume navigation seamlessly.

### System Requirements

The project must fulfill the following requirements:

1. **Vision System**: Use a camera to detect the robot's position and orientation (pose), identify obstacles, and locate the goal position.
2. **Global Path Planning**: Compute an optimal collision-free path from start to goal using a "shortest path" algorithm on a visibility graph.
3. **State Estimation**: Implement an Extended Kalman Filter (EKF) to fuse visual measurements with odometry for robust pose estimation.
4. **Motion Control**: Design a controller to follow the planned path by computing appropriate wheel velocities.
5. **Local Obstacle Avoidance**: Use the robot's proximity sensors to reactively avoid unexpected obstacles while maintaining progress toward the goal.


## System Overview and Architecture

<img src="Report/System_Overview_temporary.jpeg" alt="System Overview" width="60%">

The navigation system follows a sequential pipeline that combines global planning with real-time feedback control:

1. **Vision System** creates a map of the environment by detecting obstacles and identifying potential waypoints positioned safely away from obstacle edges where the robot can pass. This environmental map is passed to the path planner.

2. **Path Planner (Global Navigation)** receives the map and computes the shortest collision-free path from the robot's starting position to the goal using the A* algorithm on a visibility graph. The result is a sequence of waypoint coordinates that define the optimal trajectory.

3. **Kalman Filter** provides accurate state estimation by fusing two sources of information: the noisy pose measurements from the camera and the imperfect position estimates derived from wheel odometry. This sensor fusion produces a filtered pose estimate that combines the strengths of both sensors while mitigating their individual weaknesses.

4. **Motion Controller** receives the waypoint sequence from the path planner and the current filtered pose from the Kalman filter. It computes appropriate left and right wheel speeds to navigate the robot toward the next waypoint in the sequence.

5. **Local Navigation** operates in parallel with global navigation. The Thymio continuously sends its infrared proximity sensor data to the local avoidance system, which detects unexpected obstacles not visible to the camera. When obstacles are detected, the local navigation modulates the controller's wheel speeds to reactively avoid collisions while maintaining progress toward the goal.

This architecture ensures robust navigation by combining optimal global planning with reactive local obstacle avoidance.



### System Components

Each of the main components described above is implemented as a dedicated Python class, encapsulating all the functions necessary to fulfill its specific tasks. Note that the **Local Navigation** functions are integrated within the **Motion Controller** class, as they work closely together to control the robot's movement. Additionally, a **Visualizer** class provides real-time visualization on the computer screen, displaying the camera feed with overlays showing detected obstacles, robot position, planned path, and sensor readings.

The system also includes a [`utils.py`](utils.py) module that contains important constants used throughout the codebase, such as the Thymio's physical dimensions (wheel base width, wheel radius).$^{[1.1]}$

### Running the Project

To run the complete navigation system, execute the [`main.py`](main.py) script. This script contains everything needed to initialize all components, establish communication with the Thymio, start the camera feed, and execute the full navigation pipeline from start to goal.

**Note:** The entire project was designed and tested exclusively on macOS. While the core functionality should work on Windows, minor adjustments may be required to connect to the Thymio robot, for camera access, file paths, or system-specific dependencies.


### Team Contributions

The project was developed collaboratively, with each team member taking primary responsibility for specific components:

- **Daniel Alves Ataìde**: Implemented the local navigation system using the ANN approach and developed the Kalman Filter in collaboration with Rachid. Daniel also created the overall code architecture and integrated all components, ensuring that the vision system, path planner, controller, and filters worked together seamlessly.

- **Rachid Kahia**: Led the development of the vision system with contributions from the entire team, and co-developed the Kalman Filter with Daniel.

- **Nicholas Thole**: Designed and implemented the global path planning algorithm and the motion controller.

This report was primarily written by Rachid and Nicholas.

---

**Note on Citations:** Throughout this report, sources are cited using bracketed numbers (e.g. $^{[1.1]}$). All referenced sources can be found in the Bibliography section at the end of the report.

*To be able run the snippets of code in this notebook, make sure to run the following cell with all the necessary imports.*

In [11]:
# Standard library imports
import asyncio
import time

# Third-party imports
import numpy as np
import cv2
from tdmclient import ClientAsync, aw

# Project module imports
from utils import WHEEL_RADIUS_MM, THYMIO_WIDTH_MM, THYMIO2MMS
from vision_system import VisionSystem
from kalman_filter import KalmanFilter
from path_planner import PathPlanner
from motion_controller import MotionController, ThymioConnection
from visualizer import Visualizer

## Vision

A significant part of this project relies on computer vision. We made extensive use of the *OpenCV* library to implement the image-processing pipeline that enables the robot to interpret its environment. All vision-related functionalities are organized within a dedicated `VisionSystem` class, implemented in the file *vision_system.py*.

The camera was placed above the setup at a slight angle. As a first step, we calibrated the image by detecting ArUco markers positioned around the environment. These markers allowed us to isolate and crop the relevant rectangular area. We then applied a perspective transform to obtain a top-down, orthonormal view of the scene, which became the reference frame for all subsequent processing. ArUco markers were also used to detect both the robot and the goal position.

To detect obstacles, we initially considered using black objects combined with classical edge detection. However, the presence of other dark elements in the scene such as the ArUco markers and parts of the Thymio robot made this approach unreliable. We therefore switched to using blue obstacles and applied color filtering to extract blue regions from the image. This method turned out to be highly effective and was ultimately adopted in our final pipeline.

## Connecting to Thymio

### Prerequisites

Before establishing a connection in code, you must prepare the Thymio for wireless communication:

1. **Launch Thymio Suite**: Open the Thymio Suite application on your computer
2. **Detect the Robot**: Wait for your Thymio to be detected via the wireless dongle
3. **Open Aseba Studio**: Click to open Aseba Studio from Thymio Suite
4. **Unlock the Robot**: Click the unlock icon in Aseba Studio to make the robot available for external connections

This unlocking step is crucial - without it, the TDM client cannot establish a connection to the robot.

### Connection Implementation

We connect to the Thymio wirelessly using the **TDM (Thymio Device Manager) protocol** through the `tdmclient` Python library. The connection is managed by a custom [`ThymioConnection`](motion_controller.py) context manager class that ensures proper resource handling:


In [6]:
from tdmclient import ClientAsync, aw

class ThymioConnection:
    """
    Context manager for safe Thymio connection handling.
    Automatically locks on connection and unlocks on exit.
    """
    
    def __init__(self, timeout=10):
        self.timeout = timeout
        self.client = None
        self.node = None
    
    def __enter__(self):
        # Create async client
        self.client = ClientAsync()
        
        # Wait for robot to be discovered
        self.node = aw(self.client.wait_for_node())
        
        # Lock the robot for exclusive access
        aw(self.node.lock())
        
        print(f"[Thymio] Connected and locked: {self.node}")
        return self.client, self.node
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # Stop motors
        if self.node is not None:
            aw(self.node.set_variables({
                "motor.left.target": [0],
                "motor.right.target": [0],
            }))
        
        # Unlock and disconnect
        aw(self.node.stop())
        aw(self.node.unlock())
        self.client.close()
        
        print("[Thymio] Disconnected")
        return False

The connection is established at the start of the navigation loop in [`main.py`](main.py):

## Global Navigation

The global navigation system computes an optimal collision-free path from the robot's starting position to the goal before movement begins. It uses the map of static obstacles provided by the vision system to build a **visibility graph**: a network where nodes represent potential waypoints (vertices of the expanded obstacles, start, and goal), and edges connect pairs of nodes that have an unobstructed line of sight between them. The A* algorithm then searches this graph to find the shortest path, resulting in a sequence of waypoints the robot should follow.

### Visibility Graph Construction

Before constructing the visibility graph, the system first assembles a complete list of all nodes (waypoints) that will form the vertices of the graph. This node list is created by combining:

1. **Start position**: The robot's current pose (index 0 in the node list)
2. **Goal position**: The target destination (index 1 in the node list)  
3. **Expanded obstacle vertices**: All corner points of the obstacles after they have been expanded by the robot's radius to account for its physical size

This node assembly happens in the [`compute_path`](path_planner.py) method before calling the visibility graph builder:

```python
nodes = [np.array(start), np.array(goal)]
for poly in obstacles:
    for v in poly:
        nodes.append(np.array(v))
```

By placing start and goal at indices 0 and 1 respectively, the pathfinding algorithm knows exactly which nodes represent the start and end points without needing to search through the entire list.

Once the waypoint nodes are established, the system constructs a **visibility graph** that defines which waypoints can be directly connected by straight-line paths.

The visibility graph is built by the [`build_visibility_graph`](path_planner.py) method, which operates as follows:

1. **Graph Initialization**: An empty adjacency list is created for each node in the node list.

2. **Pairwise Visibility Testing**: For every unique pair of nodes, the system checks if a straight line between them is obstacle-free using the [`_is_visible`](path_planner.py) method. This visibility check includes:
   - **Special case handling**: If both nodes are vertices of the same polygon, the connection is only allowed if they are adjacent vertices (connected by an edge). This prevents diagonal connections that would cut through the obstacle's interior.
   - **Edge intersection testing**: The line segment is tested against all obstacle edges to ensure it doesn't intersect with any boundaries. The intersection test uses an **orientation-based geometric algorithm**$^{[3.1]}$ that works as follows:
     
     To determine if line segment $p_1p_2$ intersects with obstacle edge $q_1q_2$, we compute the **orientation** of four triplets of points:
     - Orientation of $(p_1, p_2, q_1)$: Is $q_1$ to the left, right, or on the line $p_1p_2$?
     - Orientation of $(p_1, p_2, q_2)$: Is $q_2$ to the left, right, or on the line $p_1p_2$?
     - Orientation of $(q_1, q_2, p_1)$: Is $p_1$ to the left, right, or on the line $q_1q_2$?
     - Orientation of $(q_1, q_2, p_2)$: Is $p_2$ to the left, right, or on the line $q_1q_2$?
     
     The orientation is computed using the **cross product** of vectors: for points $(a, b, c)$, the orientation is determined by the sign of $(b_x - a_x)(c_y - a_y) - (b_y - a_y)(c_x - a_x)$. This value is positive if $c$ is to the left of line $\overrightarrow{ab}$, negative if to the right, and zero if collinear.
     
     **Intersection occurs** when the two endpoints of each segment lie on **opposite sides** of the other segment's line. Specifically, if $(p_1, p_2, q_1)$ and $(p_1, p_2, q_2)$ have different orientations, **and** $(q_1, q_2, p_1)$ and $(q_1, q_2, p_2)$ have different orientations, then the segments intersect. Special handling is required for collinear cases where points lie on the same line.

3. **Edge Creation**: If two nodes are mutually visible (pass both checks above), an edge is added to the graph **in both directions** with a weight equal to the Euclidean distance between them.

The implementation in [`path_planner.py`](path_planner.py) demonstrates this process:

ADD A BIT OF CODE ABOVE TO BE ABLE TO RUN THIS SUCH THAT IT PRODUCES A VISIBILITY GRAPH - Call the functions from path planner

### Path Planning Algorithm

We implemented the **A\* (A-star) algorithm** to find the shortest path through the environment. A\* was chosen over simpler alternatives like Dijkstra's algorithm because it uses a **heuristic** function—an estimated cost that predicts the remaining distance from any node to the goal—to guide the search more efficiently. While Dijkstra explores nodes uniformly in all directions, A\* prioritizes nodes that appear closer to the goal based on this heuristic.$^{[3.2]}$ 

In our implementation, we use the **Euclidean distance** (straight-line distance) as the heuristic: $h(n) = \sqrt{(x_n - x_{goal})^2 + (y_n - y_{goal})^2}$. This heuristic is admissible because it never overestimates the actual shortest path distance, which is a key requirement for A\* to guarantee optimal solutions $^{[3.3]}$. This targeted exploration reduces computation time—a critical advantage for real-time robotics applications where the robot may need to replan paths quickly, such as after kidnapping detection. 

In our specific use case with only two quadrilateral obstacles (resulting in approximately 10 nodes total including start and goal), the computational difference between A\* and Dijkstra's algorithm is negligible. However, A\* was chosen for its scalability and to demonstrate best practices for path planning in more complex environments with numerous obstacles.

The A\* algorithm is implemented in the [`_a_star`](path_planner.py) method within the `PathPlanner` class, which searches the visibility graph using a priority queue to efficiently explore nodes in order of their estimated total cost to the goal.

ADD CODE HERE TO SHOW THE SHORTEST PATH WHICH IS CALCULATED

### Output
The result of the path planning is a sequence of waypoint coordinates $[(x₁, y₁), (x₂, y₂), ..., (xₙ, yₙ)]$ that defines the optimal trajectory from start to goal. This path is then passed to the motion controller, which navigates the robot through each waypoint in sequence while the Kalman filter provides accurate pose estimates.

## Control System Overview (à voir si on garde)

### Add image and explanation of the control system

## Kalman Filter

## Motion Control

The motion controller is responsible for computing wheel velocities that steer the robot toward each waypoint in the planned path. Our implementation is based on a **modified Astolfi controller**, adapted from the control law presented in the Basics of Mobile Robotics course.

### The Standard Astolfi Controller

The original Astolfi controller uses a polar coordinate representation to drive a differential-drive robot toward a target pose. It defines three error variables:
- $\rho$: the Euclidean distance to the target
- $\alpha$: the angle between the robot's heading and the direction to the target
- $\beta$: the angle of arrival (the desired final orientation at the target)

### ADD IMAGE FROM COURSE - OR OUR OWN

The control law then computes linear velocity $v$ and angular velocity $\omega$ as functions of these three variables, ensuring the robot reaches the target with a specific orientation.

### Our Modification: Ignoring $\beta$

In our application, we only care about **reaching each waypoint's position**, not about the robot's orientation upon arrival. Intermediate waypoints are simply locations the robot must pass through, and even at the final goal, the required orientation is not specified. Therefore, we **eliminate the $\beta$ term entirely** from the controller, simplifying it to:

$$\rho = \sqrt{(x_{target} - x_{robot})^2 + (y_{target} - y_{robot})^2}$$

$$\alpha = -\theta_{robot} + \text{atan2}(y_{target} - y_{robot}, x_{target} - x_{robot})$$

$$v = k_\rho \cdot \rho \cdot \cos(\alpha)$$

$$\omega = k_\alpha \cdot \alpha$$

Where $k_\rho$ and $k_\alpha$ are proportional gains (in $s^{-1}$) that control how aggressively the robot reduces distance and corrects heading errors, respectively. The function $\text{atan2}(y, x)$ is the two-argument arctangent, which computes the angle from the positive x-axis to the point $(x, y)$, correctly handling all four quadrants (unlike the standard $\arctan$ which only returns values in $[-\frac{\pi}{2}, \frac{\pi}{2}]$). The resulting angle $\alpha$ is then normalized to $[-\pi, \pi]$ to handle angle wraparound correctly.

### Conversion to Wheel Speeds

The computed linear velocity $v$ and angular velocity $\omega$ are converted to the individual left and right wheel angular velocities using the differential drive kinematics:

$$\phi_{l} = \frac{v + L \cdot \omega}{r}, \quad \phi_{r} = \frac{v - L \cdot \omega}{r}$$

Where $r$ is the wheel radius and $L$ is the half-width of the robot (distance from center to wheel). The speeds are then scaled if necessary to respect the Thymio's maximum motor speed while preserving the turning ratio.

### Why this Controller?

We chose this waypoint-based proportional controller rather than full trajectory-tracking methods—such as PID control or MPC—because it offers a good balance of simplicity, robustness, and computational efficiency. A pure PID controller is poorly suited for trajectory tracking on a differential-drive robot: the robot is nonlinear and nonholonomic, meaning it cannot directly correct lateral errors $^{[5.1]}$, yet PID implicitly assumes the system can reduce error in any direction. This mismatch often leads to oscillations or failure to converge. On the other hand, Model Predictive Control (MPC) is significantly more complex because it requires solving an optimization problem at every control step, taking into account a full motion model, prediction horizon, constraints, and numerical solver settings. This makes MPC computationally demanding and difficult to tune for real-time operation. In contrast, nonlinear controllers such as Astolfi’s law provide closed-form control inputs with minimal computation, making them considerably easier to implement

### Implementation

The controller is implemented in the [`compute_speed`](motion_controller.py) method of the `MotionController` class. For now, we consider navigation without local obstacles—the next section on **Local Navigation** explains how this controller is modulated when unexpected obstacles are detected.

The main navigation loop in [`main.py`](main.py) manages waypoint sequencing as follows:

1. **Initialization**: After path planning, the `waypoint_idx` is set to 0 (the first waypoint after the start position).

2. **Distance Check**: Each iteration, the system computes the Euclidean distance from the robot's current filtered position (from the Kalman filter) to the current waypoint.

3. **Waypoint Transition**: When this distance falls below a threshold (`WAYPOINT_THRESHOLD = 30` pixels), the waypoint index is incremented and the robot begins navigating toward the next waypoint in the sequence.

4. **Goal Detection**: When the robot reaches the final waypoint in the path (the goal) within the same 30-pixel threshold, navigation is complete. The motors are stopped and the program terminates.

5. **Speed Commands**: At each control cycle, the wheel speeds are computed by the `compute_speed` method and then sent to the robot through the `set_speed` method.

This simple proportional controller proved effective for our environment, providing smooth trajectories toward each waypoint.

## Local Navigation

While global path planning provides an optimal collision-free trajectory based on known obstacles detected by the camera, the robot must also handle **unexpected obstacles** that appear during navigation. These dynamic obstacles are not visible in the camera frame and cannot be anticipated during the planning phase. To address this, we implemented a reactive local navigation system using the Thymio's infrared proximity sensors.

### Artificial Neural Network (ANN) Approach

The local obstacle avoidance is implemented using an **Artificial Neural Network (ANN)** that runs autonomously on the Thymio at 10Hz. This approach is inspired by Braitenberg vehicles and uses a simple single-layer neural network architecture.

**Architecture:**
- **Inputs**: 7 proximity sensor readings (5 front sensors + 2 rear sensors)
- **Outputs**: 2 motor speeds (left and right wheels)
- **Weights**: A 2×7 weight matrix that maps sensor activations to motor commands

The weight matrix is designed such that:
- Front center sensors strongly reduce both wheel speeds (braking)
- Front left sensors reduce right wheel speed more than left (turn right to avoid)
- Front right sensors reduce left wheel speed more than right (turn left to avoid)
- Rear sensors provide a slight bias to help escape tight situations

**Implementation:**

The ANN program is compiled and uploaded to the Thymio using the Aseba scripting language, where it runs directly on the robot's microcontroller at 10Hz. This on-board execution was crucial for achieving **fast reaction times** to unexpected obstacles. By running the avoidance logic locally on the robot, the system operates at the Thymio's native sensor refresh rate without any communication overhead—sensor readings are processed and motor commands are updated directly on the robot. If we had instead sent sensor data to the computer for processing and waited for motor commands to be sent back wirelessly, the round-trip communication delay would significantly slow the robot's reaction to obstacles, potentially causing collisions. The local implementation eliminates this bottleneck entirely. 

The Aseba program is uploaded through the [`upload_local_avoidance`](motion_controller.py) method in the `MotionController` class:

In [7]:
def upload_local_avoidance(self, node, threshold=2000):
    """Upload local obstacle avoidance program to Thymio"""
    program = """
    var max_prox
    var i
    var w_l[7] = [40, 20, -20, -20, -40, 30, -10]
    var w_r[7] = [-40, -20, -20, 20, 40, -10, 30]
    var x[7]
    var y[2]
    
    onevent prox
        # Scale sensor readings
        for i in 0:6 do
            x[i] = prox.horizontal[i] / 200
        end
        
        # Find max proximity
        max_prox = 0
        for i in 0:6 do
            if prox.horizontal[i] > max_prox then
                max_prox = prox.horizontal[i]
            end
        end
        
        # Only activate if obstacle detected
        if max_prox > """ + str(threshold) + """ then
            # Compute weighted outputs
            y[0] = 0
            y[1] = 0
            for i in 0:6 do
                y[0] = y[0] + w_l[i] * x[i]
                y[1] = y[1] + w_r[i] * x[i]
            end
            
            motor.left.target = y[0]
            motor.right.target = y[1]
        end
    """
    
    error = node.compile(program)
    if error is not None:
        print(f"Compilation error: {error['error_msg']}")
    else:
        node.run()
        return True

The Aseba program runs at the sensor event rate (`onevent prox`), which triggers at 10Hz whenever the proximity sensors update. The program:

1. **Scales sensor readings** by dividing by 200 to normalize inputs
2. **Finds maximum proximity** across all sensors to determine if an obstacle is present
3. **Threshold check**: Only activates avoidance if `max_prox > threshold` (which we set at 2000)
4. **Computes weighted outputs**: For each wheel, calculates the sum of weighted sensor inputs
5. **Updates motor targets**: Directly sets `motor.left.target` and `motor.right.target`

### Integration with Global Navigation

The local avoidance system operates **in parallel** with the global motion controller:

- When no obstacles are detected (below threshold), the ANN remains inactive and the robot follows the global path planned by the motion controller
- When an obstacle is detected, the ANN takes over and modulates the motor speeds to avoid the obstacle
- Once the obstacle is cleared, control seamlessly returns to the waypoint-based navigation

This hybrid approach provides the **robustness of reactive control** for handling unexpected situations while maintaining the **optimality of planned navigation** when the environment matches expectations. The 10Hz on-board reaction time ensures the robot can respond quickly enough to avoid collisions even when moving at higher speeds.

### Why ANN Instead of a State Machine?

We chose the ANN approach over a traditional state machine for several reasons. A state machine would require explicitly defining states (e.g., "obstacle detected left," "obstacle detected center," "clear path") and hard-coded transitions between them, along with predefined motor responses for each state. This approach becomes complex and difficult to tune as the number of sensor combinations grows—with 7 sensors, the state space can become unwieldy. In contrast, the ANN provides a **continuous, smooth response** that naturally handles all sensor configurations through weighted combinations. The weights can be tuned experimentally by observing the robot's behavior, and the network inherently interpolates between extreme cases without requiring explicit state definitions. Additionally, the ANN implementation in Aseba is simpler and more compact than a multi-state conditional logic structure, making it easier to upload, debug, and modify. This continuous control paradigm also avoids abrupt transitions between discrete behaviors, resulting in smoother obstacle avoidance trajectories.



## Kidnapping Resilience

The **kidnapping problem** refers to the scenario where the robot is suddenly moved to a different location without its knowledge—for example, if someone picks up the robot and places it elsewhere in the environment. This is a critical test of a navigation system's robustness, as the robot's internal belief about its position becomes drastically incorrect.

### Detection Condition

We detect kidnapping by monitoring the **predicted position change** between consecutive Kalman filter updates. At each control cycle iteration, we compare the difference in position of the robot:

$$\Delta_{position} = \|\mathbf{x}_{k} - \mathbf{x}_{k-1}\|$$

Where $\mathbf{x}_{k}$ is the current filtered position and $\mathbf{x}_{k-1}$ is the position from the previous iteration. If this displacement exceeds a threshold of **100 pixels**, we conclude that the robot has been kidnapped. This threshold was chosen to be large enough to avoid false positives from normal motion or sensor noise, but small enough to detect genuine kidnapping events quickly.

The key insight is that normal robot motion is continuous and constrained by the robot's maximum speed. A sudden jump of 100 pixels (corresponding to about 11cm in the real environment) in a single control cycle (~0.1 seconds) is physically impossible through normal driving, and thus indicates external intervention.

### Recovery Procedure

When kidnapping is detected, the system immediately:

1. **Stops incrementing the waypoint index**: The robot's current waypoint target is no longer valid
2. **Replans the path**: Calls `compute_path` using the robot's new filtered position (provided by the Kalman filter after incorporating the camera measurement) and the original goal position
3. **Resets waypoint tracking**: Sets `waypoint_idx = 0` to begin following the new path from its first waypoint

This recovery is completely automatic and requires no manual intervention. The robot seamlessly transitions from following the old path to following a newly computed optimal path from its new location.

### Implementation

The kidnapping detection and recovery logic is implemented in the main navigation loop in [`main.py`](main.py):


In [ ]:
last_state = kalman.state[0:2].copy()

# Kalman filter
kalman.predict(current_speed_px, dt)
kalman.update(robot_pose)

##...

# Kidnapping check
if np.linalg.norm(kalman.state[0:2] - last_state) > 100:
    path = planner.compute_path(kalman.state[0:2], vision.goal_position, obstacles)
    waypoint_idx = 0

The code saves the robot's predicted position before the Kalman update step, then compares it to the filtered position after the update. The large difference (>100 pixels) indicates that kidnapping has occured.

The `compute_path` method in the `PathPlanner` class then efficiently recalculates the optimal path from the new position. Since the A* algorithm is very fast for our environment, this replanning happens in milliseconds and the robot can immediately begin following the new trajectory.

This approach provides **seamless recovery** from kidnapping events with minimal disruption to navigation. The robot doesn't need to stop, return to a known location, or perform any special recovery maneuver—it simply computes a new path and continues toward the goal from wherever it finds itself.

## Dead Reckoning

**Dead reckoning** refers to the ability to estimate position based solely on motion measurements, without external reference points. In our system, this means navigating using only wheel odometry when the camera is temporarily unavailable.

While we demonstrated this capability by intentionally hiding the camera during testing, this implementation provides valuable robustness for real-world scenarios where vision loss occurs unintentionally. In practical deployments, the camera could be accidentally obstructed by objects moving through the workspace, lighting conditions could temporarily change (shadows, reflections, sudden brightness variations), or the robot's marker might be partially occluded by unexpected elements in the environment. By handling vision loss gracefully through dead reckoning, the system remains operational during these disruptions rather than failing or requiring manual intervention.

### Handling Vision Loss

Our navigation system is designed to continue operating even when visual measurements are lost. The Kalman filter architecture naturally handles this scenario through its two-step process:

1. **Prediction step**: Uses the differential drive motion model and wheel speed measurements to predict the robot's new position based on its previous state and control inputs
2. **Update step**: Incorporates camera measurements to correct the prediction—but can be skipped if no measurement is available

When the camera cannot detect the robot (i.e., `robot_pose = None`), the Kalman filter's update method detects this and continues with **prediction-only mode**:


In [ ]:
def update(self, measurement):
    """
    Correction step with vision measurement.
    
    Args:
        measurement: np.array [x, y, theta] from vision, or None if not detected
    """
    if measurement is None:
        # Increase uncertainty when no measurement available
        self.P += self.Q * 2
        return
    
    # ... normal Kalman update with measurement ...

When `measurement is None`, the filter:
- **Skips the correction step**: No innovation or Kalman gain is computed
- **Increases uncertainty**: The covariance matrix `P` is inflated to reflect growing uncertainty about the robot's true position
- **Relies solely on odometry**: The predicted state from wheel encoders becomes the robot's best position estimate

### Navigation During Vision Loss

During periods without vision, the robot continues to follow the planned waypoint sequence using the odometry-based pose estimate and compute control commands based on the predicted state. The system makes no special accommodation for vision loss—the same control loop continues running, simply using the prediction-only state estimate. This seamless fallback is a key advantage of the Kalman filter architecture: it naturally degrades gracefully rather than failing completely.

### Limitations and Recovery

**Precision degradation**: Without vision corrections, odometry errors accumulate over time due to several factors:
- Wheel slippage on the surface
- Estimation errors from numerical integration of wheel velocities over discrete time steps
- Imperfect physical parameters such as wheel diameter and baseline measurements (e.g., `THYMIO2MMS` conversion factor)

The uncertainty in the robot's position grows continuously during vision loss, reflected in the expanding covariance ellipse in our demo video. However, for short periods (a few seconds), dead reckoning provides sufficient accuracy to maintain progress toward the goal.

**Automatic recovery**: As soon as the camera detects the robot again, the Kalman filter's update step resumes, immediately correcting the accumulated drift and reducing uncertainty. The robot seamlessly transitions back to high-precision navigation without requiring any special recovery procedure.

This dead reckoning capability demonstrates the system's **resilience to computer vision failures**, ensuring that temporary vision loss does not cause the robot to stop or lose its way—it simply continues with the best available information until more accurate measurements become available again.

## Conclusion

This project successfully demonstrated a complete autonomous navigation system integrating computer vision, path planning, sensor fusion, motion control, and reactive obstacle avoidance. The Thymio robot navigated reliably from start to goal while handling both static obstacles detected by the overhead camera and dynamic obstacles sensed by its proximity sensors. The system's robustness was validated through two critical challenges: seamless recovery from kidnapping events through automatic path replanning, and continued navigation during vision loss using dead reckoning with wheel odometry.

The integration of global and local navigation strategies proved particularly effective. The A* algorithm on visibility graphs provided optimal collision-free paths, while the modified Astolfi controller ensured smooth waypoint following. The Kalman filter's sensor fusion elegantly combined noisy camera measurements with imperfect odometry, providing accurate state estimates even when one sensor modality temporarily failed. The ANN-based local avoidance, running autonomously on the Thymio at 10Hz, enabled fast reactive responses to unexpected obstacles without communication delays.

It is worth noting that this project is typically assigned to groups of four students, yet our team of three successfully completed all required objectives and delivered a fully functional system. While the reduced team size presented additional workload challenges, it also fostered closer collaboration and deeper individual understanding of each system component. The experience of designing, implementing, debugging, and integrating multiple subsystems provided invaluable hands-on learning in robotics, control theory, computer vision, and software engineering.

We would like to express our sincere gratitude to **Professor Francesco Mondada** for providing this comprehensive and engaging course that bridged theoretical concepts with practical implementation. The project structure allowed us to experience the full pipeline of mobile robotics development, from low-level sensor integration to high-level decision-making algorithms. We also extend our thanks to the **teaching assistants** who guided us during the initial stages of the project, helping us navigate the technical setup, understand the Thymio platform, and debug early implementation challenges. Their support was instrumental in establishing a solid foundation for our work.

This project has been both challenging and rewarding, offering us a genuine appreciation for the complexity of autonomous systems and the importance of robust, well-integrated software architecture in robotics applications.

## Bibliography

**Thymio data**

[1.1]

**Vision**

**Path Planning**

[3.1] GeeksforGeeks, “Check if two given line segments intersect.” Accessed: Dec. 3, 2025. [Online]. Available: https://www.geeksforgeeks.org/check-if-two-given-line-segments-intersect/

[3.2] Baeldung, “Dijkstra vs. A* – Pathfinding,” Baeldung on Computer Science, 2023.
Available: https://www.baeldung.com/cs/dijkstra-vs-a-pathfinding

[3.3] K. Kask, “Informed Heuristic Search,” CS 271: Introduction to Artificial Intelligence, University of California, Irvine. Accessed: Dec. 3, 2025. [Online]. Available: https://ics.uci.edu/~kkask/Fall-2016%20CS271/slides/03-InformedHeuristicSearch.pdf

**Kalman Filter**

**Controller**

[5.1] I. Anvari, “Non-holonomic Differential Drive Mobile Robot Control & Design: Critical Dynamics and Coupling Constraints.” M.S. thesis, Univ. of Leeds, Leeds, U.K., 2015.

**Local Avoidance**

Source for the value of 10Hz
**Dead Reckoning**

**Other**